In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re
import math
import os
import ast

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, '크롤링')
original_table_path = os.path.join(data_dir, r'원본(크롤링)\original_NE.csv')
dron_park_path = os.path.join(data_dir, r'원본(크롤링)\df_dron_park.csv')
original_table = pd.read_csv(original_table_path, encoding='utf-8')
dron_park = pd.read_csv(dron_park_path, encoding='utf-8')
# original_table = pd.read_csv(r"C:\Users\user\Desktop\python 정리집\웹크롤링 팀프로젝트\original_NE.csv", encoding='utf-8')
# dron_park = pd.read_csv(r"C:\Users\user\Desktop\python 정리집\웹크롤링 팀프로젝트\df_dron_park.csv", encoding='utf-8')
print(original_table)
print(dron_park)

      구역 코드   위치                                               수평범위  \
0      UA 2  구성산  Circle with radius of 1.8 km (1.0 NM) centered...   
1      UA 3   약산  Circle with radius of 0.7 km (0.4 NM) centered...   
2      UA 4  봉화산  Circle with radius of 4.0 km (2.2 NM) centered...   
3      UA 5  덕두산  Circle with radius of 4.5 km (2.4 NM) centered...   
4      UA 6   금산  Circle with radius of 2.1 km (1.1 NM) centered...   
5      UA 7   홍산  Circle with radius of 1.2 km (0.7 NM) centered...   
6      UA 9   양평  373010N 1272300E - 373010N 1273200E - 372700N ...   
7     UA 10   고창  Circle with radius of 4.0 km (2.2 NM) centered...   
8     UA 14   공주  363038N 1270033E - 363002N 1270713E - 362604N ...   
9     UA 19  시화호  371751N 1264215E - 371724N 1265000E - 371430N ...   
10    UA 21  방장산  Circle with radius of 3.0 km (1.6 NM) centered...   
11    UA 24   구좌  Circle with radius of 2.8 km (1.5 NM) centered...   
12    UA 25   하동  350147N 1274325E - 350145N 1274741E - 345915N ...   
13    

In [3]:
def dms_to_dd(dms_str):
    m = re.match(r"(\d{2,3})(\d{2})(\d{2})([NSEW])", dms_str.replace(" ", ""))
    if not m:
        return None
    deg, minute, second, direction = m.groups()
    dd = int(deg) + int(minute) / 60 + int(second) / 3600
    if direction in ["S", "W"]:
        dd *= -1
    return dd

In [4]:
def extract_circle_data(text, region):
    m = re.search(
        r'Circle with radius of ([\d.]+)\s*(km|m).*?centered on\s*([\d]{6,7}[NSEW])\s*([\d]{6,7}[NSEW])',
        text
    )
    if m:
        radius = float(m.group(1))
        unit = m.group(2).lower()
        if unit == "m":
            radius /= 1000
        lat_str = m.group(3)
        lon_str = m.group(4)
        # 지역명이 "광주"인 경우에만 경도 교정 (예: 128... → 126...)
        if region and "광주" in region:
            if lon_str.startswith("128"):
                lon_str = "126" + lon_str[3:]
        return radius, dms_to_dd(lat_str), dms_to_dd(lon_str)
    return None, None, None

In [5]:
def build_circle_table():
    df = original_table
    df['반경'] = None
    df['위도'] = None
    df['경도'] = None
    for idx, row in df.iterrows():
        text = row['수평범위']
        region = row['위치']  # 여기서는 '위치' 열에 지역명이 들어 있다고 가정
        if text and "Circle" in text:
            radius, lat, lon = extract_circle_data(text, region)
            df.at[idx, '반경'] = radius
            df.at[idx, '위도'] = lat
            df.at[idx, '경도'] = lon
    return df[df['반경'].notnull()][["구역 코드", "위치", "반경", "위도", "경도"]].reset_index(drop=True)

In [6]:
circle_table = build_circle_table()
print(circle_table)

      구역 코드   위치    반경         위도          경도
0      UA 2  구성산   1.8  35.739167    127.0075
1      UA 3   약산   0.7  35.739167  128.417222
2      UA 4  봉화산   4.0  35.625278  129.092222
3      UA 5  덕두산   4.5  35.411389    127.5325
4      UA 6   금산   2.1  34.736389  127.981111
5      UA 7   홍산   1.2  35.828056  127.081111
6     UA 10   고창   4.0  35.386389  126.731389
7     UA 21  방장산   3.0  35.449444  126.738056
8     UA 24   구좌   2.8  33.478056  126.822778
9     UA 27  미악산   1.2       33.3  126.554444
10    UA 28  서운산   2.0  36.930556  127.283056
11    UA 29   오천   2.0  36.953056  127.287778
12    UA 30   북좌   2.0     37.045  127.327778
13  * UA 32   퇴촌   0.3  37.466667    127.3025
14  * UA 40   고령  0.08  35.842778  128.444167
15  **UA 42   광주  0.05  35.221667  126.861667


In [7]:
# "드론공원"까지의 부분만 남기는 함수
def extract_location_name(region):
    if pd.isnull(region):
        return region
    if "드론공원" in region:
        return region.split("드론공원")[0] + "드론공원"
    return region

In [8]:
# 좌표 컬럼이 문자열 형태로 저장되어 있다면 ast.literal_eval로 리스트로 변환
dron_park["좌표"] = dron_park["좌표"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# '지역' 컬럼의 값을 '위치' 컬럼으로 재설정 (드론공원까지의 부분만 남김)
dron_park["위치"] = dron_park["지역"].apply(extract_location_name)

In [9]:
# Circle: 좌표 목록의 길이가 1인 경우 (예: 광주 북구 영산강변 드론공원 등)
df_Circle_new = dron_park[dron_park["좌표"].apply(lambda x: isinstance(x, list) and len(x) == 1)].copy()
# 좌표 목록의 단일 쌍에서 위도와 경도를 분리
df_Circle_new["위도"] = df_Circle_new["좌표"].apply(lambda x: x[0][0] if isinstance(x, list) and len(x)==1 else None)
df_Circle_new["경도"] = df_Circle_new["좌표"].apply(lambda x: x[0][1] if isinstance(x, list) and len(x)==1 else None)
# 만약 반경 데이터가 df_dron에 있다면 사용, 없으면 None 처리 (여기서는 None으로 처리)
df_Circle_new["반경"] = None 
df_Circle_new['구역 코드'] = None
df_Circle_new = df_Circle_new[["구역 코드", "위치", "반경", "위도", "경도"]]
print(df_Circle_new)

  구역 코드               위치    반경         위도          경도
1  None  광주 북구 영산강변 드론공원  None  35.221667  126.861667
2  None     광나루 한강변 드론공원  None  37.546639  127.120528


In [10]:
final_circle = pd.concat([circle_table, df_Circle_new])

In [11]:
print(final_circle)

      구역 코드               위치    반경         위도          경도
0      UA 2              구성산   1.8  35.739167    127.0075
1      UA 3               약산   0.7  35.739167  128.417222
2      UA 4              봉화산   4.0  35.625278  129.092222
3      UA 5              덕두산   4.5  35.411389    127.5325
4      UA 6               금산   2.1  34.736389  127.981111
5      UA 7               홍산   1.2  35.828056  127.081111
6     UA 10               고창   4.0  35.386389  126.731389
7     UA 21              방장산   3.0  35.449444  126.738056
8     UA 24               구좌   2.8  33.478056  126.822778
9     UA 27              미악산   1.2       33.3  126.554444
10    UA 28              서운산   2.0  36.930556  127.283056
11    UA 29               오천   2.0  36.953056  127.287778
12    UA 30               북좌   2.0     37.045  127.327778
13  * UA 32               퇴촌   0.3  37.466667    127.3025
14  * UA 40               고령  0.08  35.842778  128.444167
15  **UA 42               광주  0.05  35.221667  126.861667
1      None  광

In [19]:
location_map = {
    '구성산': '전주(구성산)',
    '약산': '대구(약산)',
    '봉화산': '울산(봉화산)',
    '덕두산': '남원(덕두산)',
    '홍산': '전주(홍산)',
    '방장산': '광주(방장산)',
    '구좌': '제주(구좌)',
    '미악산': '제주(지악산)',
    '서운산': '천안(서운산)',
    '오천': '천안(오천)',
    '북좌': '천안(북좌)',
    '퇴촌': '양평(퇴촌)',
    '고령': '대구(고령)',
    '광주 북구 영산강변 드론공원': '광주(광주 북구 영산강변 드론공원)',
    '광나루 한강변 드론공원': '서울(광나루 한강변 드론공원)'
}

# '위치' 컬럼에 있는 값들을 매핑된 값으로 바꿔줍니다.
final_circle['위치'] = final_circle['위치'].map(location_map).fillna(final_circle['위치'])

final_circle["지역"] = final_circle["위치"].str[:2]

# 결과 확인
print(final_circle)

      구역 코드                   위치    반경         위도          경도  지역
0      UA 2              전주(구성산)   1.8  35.739167    127.0075  전주
1      UA 3               대구(약산)   0.7  35.739167  128.417222  대구
2      UA 4              울산(봉화산)   4.0  35.625278  129.092222  울산
3      UA 5              남원(덕두산)   4.5  35.411389    127.5325  남원
4      UA 6                   금산   2.1  34.736389  127.981111  금산
5      UA 7               전주(홍산)   1.2  35.828056  127.081111  전주
6     UA 10                   고창   4.0  35.386389  126.731389  고창
7     UA 21              광주(방장산)   3.0  35.449444  126.738056  광주
8     UA 24               제주(구좌)   2.8  33.478056  126.822778  제주
9     UA 27              제주(지악산)   1.2       33.3  126.554444  제주
10    UA 28              천안(서운산)   2.0  36.930556  127.283056  천안
11    UA 29               천안(오천)   2.0  36.953056  127.287778  천안
12    UA 30               천안(북좌)   2.0     37.045  127.327778  천안
13  * UA 32               양평(퇴촌)   0.3  37.466667    127.3025  양평
14  * UA 4

In [12]:
data_dir = os.path.join(parent_dir, '데이터')
original_table_path = os.path.join(data_dir, r'전처리\Circle_table.csv')
circle_table.to_csv(original_table_path, encoding='cp949', index=False)